In [11]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [12]:
test_df = pd.read_excel('테스트 셋_800 완료.xlsx')

In [13]:
test_df.head()

,관리번호,설비번호(구),설비명,수리일자,고장내용,고장부품,불량유형,조치내용,수리처,수리시간(분),수리금액
0,Z000000001,Z000000001,기타,2023-01-03,3공장렉 철거후 2공장으로 이동,3공장 렉,철거,2공장으로 이동,오정수이승희,480,NaN
1,Z000000001,Z000000001,기타,2023-01-03,CN7라인 천정 전등 교체,CN7라인 천정 전등,고장,전등 교체,송명호윤동태,180,NaN
2,Z000000001,Z000000001,기타,2023-01-03,E4L 인너카바 리벳툴 진공이상 리벳 잔영물 제거 청소,E4L 인너카바 리벳툴,진공이상,리벳 잔영물 제거 청소,최혁진윤동태,30,NaN
3,Z000000001,Z000000001,기타,2023-01-03,전착도장 대차 제작중,전착도장 대차,NaN,제작중,김정표오정수,140,NaN
4,M000000038,ASM-029,조립기(DH 스위치 조립기),2023-01-04,#40 LH 트램스파 동작이상 전후진 솔밸브 단선 수리,#40 LH 트램스파 전후진 솔밸브,동작이상,단선 수리,송명호윤동태,150,NaN


In [14]:
def format_data(sample):
    # 고장부품, 불량유형, 조치내용을 딕셔너리 형태로 구성
    response_dict = {
        "고장부품": sample["고장부품"],
        "불량유형": sample["불량유형"], 
        "조치내용": sample["조치내용"]
    }
    
    return {
        "messages": [
            {
                "role": "system",
                "content": '''주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.''',
            },
            {
                "role": "user",
                "content": f"고장 내용: {sample['고장내용']}",
            },
            {
                "role": "assistant",
                "content": str(response_dict)  # 나머지 3개 필드를 딕셔너리로 변환하여 assistant 응답으로
            },
        ],
    }

In [15]:
test_dataset = []
for _, row in test_df.iterrows():
    test_dataset.append(format_data(row))

print(f"\n테스트 데이터 개수: Test {len(test_dataset)}개")


테스트 데이터 개수: Test 800개


In [16]:
test_dataset = Dataset.from_list(test_dataset)

In [17]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [18]:
print(prompt_lst[42])

<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 생산수량 간헐적 수량 누락 카운터 단자 재 조임<|im_end|>
<|im_start|>assistant


In [19]:
print(label_lst[42])


{'고장부품': '카운터 단자', '불량유형': '생산 수량 간헐적 누락', '조치내용': '카운터 단자 재 조임'}<|im_end|>



In [2]:
base_model_id = "iamjoon/Qwen-StructureParser"

model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [20]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [22]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [23]:
for prompt, label in zip(prompt_lst[20:25], label_lst[20:25]):
    # print(f"    prompt:\n{prompt}")
    print(f"    input:\n{prompt}")
    print("-"*50)
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("="*50)

    input:
<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 전착행거 대차 제작<|im_end|>
<|im_start|>assistant
--------------------------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train 

    response:
{'고장부품': '전착행거 대차', '불량유형': '제작 불량', '조치내용': '대차 제작'}
    label:

{'고장부품': '전착행거 대차', '불량유형': '제작불량', '조치내용': '전착행거 대차 제작'}<|im_end|>

    input:
<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 파렛트 바퀴 수리,잠금고리 수리<|im_end|>
<|im_start|>assistant
--------------------------------------------------
    response:
{'고장부품': '파렛트 바퀴', '불량유형': '수리 필요', '조치내용': '바퀴 수리, 잠금고리 수리'}
    label:

{'고장부품': '파렛트 바퀴, 잠금고리', '불량유형': '고장', '조치내용': '바퀴, 잠금고리 수리'}<|im_end|>

    input:
<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Dictionary 형태로 반환하시오.<|im_end|>
<|im_start|>user
고장 내용: 코킹감지센서 단선 수리<|im_end|>
<|im_start|>assistant
--------------------------------------------------
    response:
{'고장부품': '코킹감지센서', '불량유형': '단선', '조치내용': '센서 수리'}
    label:

{'고장부품': '코킹감지센서', '불량유형': '단선', '조치내용': '센서 수리'}<|im_end|>

    input:
<|im_start|>system
주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 추출하시오.
이때 파이썬의 Diction